In [1]:
# 콘텐츠(추천) 기반 필터링

# 협업 필터링

# - 최근접 이웃(사용자) 기반 협업필터링 - 사용자 평점 매트릭스
#    - 사용자 기반
#    - 아이템 기반 : 더 많이 사용.. 

# - 잠재요인 협업 필터링 - 행렬 변환

In [2]:
# 사용자 평점 
# 명시적 : 영화평점, 상품 평점, 좋아요, 리뷰..
# 암시적 : 조회, 구매, 방문수, 머문 시간..

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
movies = pd.read_csv('../data_files/movie.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  27278 non-null  int64 
 1   title    27278 non-null  object
 2   genres   27278 non-null  object
dtypes: int64(1), object(2)
memory usage: 639.5+ KB


In [6]:
ratings = pd.read_csv('../data_files/rating.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [7]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 610.4+ MB


In [8]:
ratings.tail()

,userId,movieId,rating,timestamp
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24
20000262,138493,71619,2.5,2009-10-17 20:25:36


### 사용자 기반

In [9]:
merge_df = pd.merge(movies, ratings, on='movieId')
merge_df

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.5,2009-01-02 01:13:41
...,...,...,...,...,...,...
20000258,131254,Kein Bund für's Leben (2007),Comedy,79570,4.0,2015-03-30 19:32:59
20000259,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,79570,4.0,2015-03-30 19:48:08
20000260,131258,The Pirates (2014),Adventure,28906,2.5,2015-03-30 19:56:32
20000261,131260,Rentun Ruusu (2001),(no genres listed),65409,3.0,2015-03-30 19:57:46


In [10]:
merge_df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.5,2009-01-02 01:13:41


In [11]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000263 entries, 0 to 20000262
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   title      object 
 2   genres     object 
 3   userId     int64  
 4   rating     float64
 5   timestamp  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 1.0+ GB


In [167]:
ratings = merge_df[['movieId', 'userId', 'rating', 'title']]
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column   Dtype  
---  ------   -----  
 0   movieId  int64  
 1   userId   int64  
 2   rating   float64
 3   title    object 
dtypes: float64(1), int64(2), object(1)
memory usage: 762.9+ MB


In [168]:
ratings = ratings.sample(10000)

In [99]:
ratings['movieId'] = ratings['movieId'].astype('int32')
ratings['userId'] = ratings['userId'].astype('int32')
ratings['rating'] = ratings['rating'].astype('int32')

In [169]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 2672090 to 8550621
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  10000 non-null  int64  
 1   userId   10000 non-null  int64  
 2   rating   10000 non-null  float64
 3   title    10000 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 390.6+ KB


In [170]:
ratings_matrix = ratings.pivot_table(index='userId', columns='title', values = 'rating')

In [171]:
ratings_matrix.shape

(8922, 3314)

In [18]:
import pickle

In [19]:
with open('ratings_matrix.pickle', 'wb') as f:
    pickle.dump(ratings_matrix, f)

In [20]:
with open('ratings_matrix.pickle', 'rb')  as f:
    ratings_matrix = pickle.load(f)

In [173]:
# 사용자 평점 메트릭스 - 사용자 기반 추천 시스템
# sparse matrix - 사용자가 직접 본 영화의 수는 제한적이다..

ratings_matrix

title,'Round Midnight (1986),"'burbs, The (1989)",(500) Days of Summer (2009),1-900 (06) (1994),10 (1979),10 Things I Hate About You (1999),100 Rifles (1969),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),...,Zeitgeist: Addendum (2008),Zelig (1983),Zero Effect (1998),Zombie Strippers! (2008),Zombieland (2009),Zoolander (2001),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [174]:
ratings_matrix.fillna(0, inplace=True)
ratings_matrix.index = list(range(len(ratings_matrix.index)))
ratings_matrix.head()

title,'Round Midnight (1986),"'burbs, The (1989)",(500) Days of Summer (2009),1-900 (06) (1994),10 (1979),10 Things I Hate About You (1999),100 Rifles (1969),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),...,Zeitgeist: Addendum (2008),Zelig (1983),Zero Effect (1998),Zombie Strippers! (2008),Zombieland (2009),Zoolander (2001),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
ratings_matrix.shape

(8924, 3378)

In [112]:
# 고객님과 유사한 사용자가 다음의 콘텐츠도 구매하였습니다..

In [113]:
from sklearn.metrics.pairwise import cosine_similarity
user_sim = cosine_similarity(ratings_matrix)  # row 간에 cosine 유사도를 계산
user_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [114]:
user_sim.shape

(8924, 8924)

In [115]:
user_sim_sort = user_sim.argsort(axis=1)[:, ::-1]
user_sim_sort

array([[   0, 3917, 1544, ..., 5956, 5957, 4461],
       [1035, 4973,    1, ..., 5955, 5956,    0],
       [  71,  865, 6666, ..., 5956, 5957,    0],
       ...,
       [3866, 8921, 2970, ..., 5956, 5957,    0],
       [8922, 6565, 3471, ..., 5956, 5957,    0],
       [8923, 8615, 7041, ..., 5955, 5956,    0]], dtype=int64)

In [119]:
user_sim_sort[10][1:11]  # 유사도가 높은 사용자 상위 10

array([7634,  776, 8724, 6495, 7415,  254, 4283, 6802, 8104, 6152],
      dtype=int64)

In [121]:
# 유사도가 높은 사용자 상위 10명의 영화 평점 데이터

df = ratings_matrix.loc[user_sim_sort[10][1:11]]
df

movieId,1,2,3,4,5,6,7,9,10,11,...,114552,114662,114795,115122,115149,115220,118696,118900,121227,125914
7634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
776,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8724,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
# 10번 사용자와 유사도가 높은 사용자들의 영화평점 순위

df.sum().sort_values(ascending=False)[:5]

movieId
364     38.0
5955     0.0
5890     0.0
5898     0.0
5902     0.0
dtype: float64

In [124]:
user_sim[10].argsort()[::-1]

array([3778, 7634,  776, ..., 5951, 5952,    0], dtype=int64)

In [131]:
def seen_movies(user_id):
    movie_list = ratings_matrix.iloc[user_id]
    seen_movies = movie_list[ratings_matrix.iloc[user_id] > 0]
    return seen_movies.index

In [132]:
seen_movies(5)

Int64Index([1204], dtype='int64', name='movieId')

In [150]:
seen_movie_list = seen_movies(30)
    
user_sim_sort = user_sim[30].argsort()[::-1][:10 * 5]
df = ratings_matrix.loc[user_sim_sort]

# unseen_movie_list = set(df.index).difference(set(seen_movie_list))
unseen_movie_list

{30,
 2932,
 2933,
 2934,
 2935,
 2936,
 2937,
 2938,
 2939,
 2940,
 2941,
 2942,
 2943,
 2961,
 2962,
 2963,
 2964,
 2965,
 2966,
 2967,
 2968,
 2969,
 2970,
 2971,
 2972,
 2973,
 2974,
 2975,
 2976,
 2977,
 2978,
 2979,
 2980,
 2981,
 2982,
 2983,
 2984,
 2985,
 2986,
 2987,
 2988,
 2989,
 2990,
 2991,
 2992,
 2993,
 2994,
 2995,
 2997,
 8923}

In [127]:
def sim_user_based_recomm(user_sim, ratings_matrix, user_id, n_top=10):
    
    seen_movie_list = seen_movies(user_id)
    
    user_sim_sort = user_sim[user_id].argsort()[::-1][:n_top * 5]
    df = ratings_matrix.loc[user_sim_sort]
    
    unseen_movie_list = set(df.index).difference(set(seen_movie_list))
    
    df = df.loc[unseen_movie_list]
    
    return df.sum().sort_values(ascending=False)    

In [128]:
sim_user_based_recomm(user_sim, ratings_matrix, 30, n_top=10)

movieId
2959      8.0
2944      5.0
1273      5.0
1393      5.0
1380      5.0
         ... 
2322      0.0
2324      0.0
2327      0.0
2328      0.0
125914    0.0
Length: 3378, dtype: float64

### 아이템 기반 이웃 협업 필터링..

In [175]:
ratings_matrix[:5]

title,'Round Midnight (1986),"'burbs, The (1989)",(500) Days of Summer (2009),1-900 (06) (1994),10 (1979),10 Things I Hate About You (1999),100 Rifles (1969),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),...,Zeitgeist: Addendum (2008),Zelig (1983),Zero Effect (1998),Zombie Strippers! (2008),Zombieland (2009),Zoolander (2001),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [176]:
# 사용자 평점 기반 영화들 간의 유사도 계산
# 사용자 평점을 기반으로 영화유사도를 계산

In [177]:
item_sim = cosine_similarity(ratings_matrix.T, ratings_matrix.T)
item_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [178]:
item_sim.shape

(3314, 3314)

In [179]:
item_sim_df = pd.DataFrame(item_sim, index=ratings_matrix.columns, columns =ratings_matrix.columns )
item_sim_df.head(3)

title,'Round Midnight (1986),"'burbs, The (1989)",(500) Days of Summer (2009),1-900 (06) (1994),10 (1979),10 Things I Hate About You (1999),100 Rifles (1969),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),...,Zeitgeist: Addendum (2008),Zelig (1983),Zero Effect (1998),Zombie Strippers! (2008),Zombieland (2009),Zoolander (2001),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,
'Round Midnight (1986),1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"'burbs, The (1989)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(500) Days of Summer (2009),0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [180]:
# 사용자 평점 기준으로 유사한 영화를 추천..

item_sim_df['Godfather, The (1972)'].sort_values(ascending=False)[:10]

title
Godfather, The (1972)                                  1.000000
Hook (1991)                                            0.090789
Mary Poppins (1964)                                    0.077237
'Round Midnight (1986)                                 0.000000
P.S. I Love You (2007)                                 0.000000
Outbreak (1995)                                        0.000000
Outfoxed: Rupert Murdoch's War on Journalism (2004)    0.000000
Outland (1981)                                         0.000000
Outlaw Josey Wales, The (1976)                         0.000000
Outlaw, The (1943)                                     0.000000
Name: Godfather, The (1972), dtype: float64

In [181]:
item_sim_df["Schindler's List (1993)"].sort_values(ascending=False)[1:10]

title
Tomorrow Never Dies (1997)                             0.067531
'Round Midnight (1986)                                 0.000000
Over the Hedge (2006)                                  0.000000
Out of Africa (1985)                                   0.000000
Out of Sight (1998)                                    0.000000
Outbreak (1995)                                        0.000000
Outfoxed: Rupert Murdoch's War on Journalism (2004)    0.000000
Outland (1981)                                         0.000000
Outlaw Josey Wales, The (1976)                         0.000000
Name: Schindler's List (1993), dtype: float64

In [ ]:
# 영화의 유사도와 사용자 평점을 함께 고려한 영화 추천

In [182]:
ratings_matrix.head()

title,'Round Midnight (1986),"'burbs, The (1989)",(500) Days of Summer (2009),1-900 (06) (1994),10 (1979),10 Things I Hate About You (1999),100 Rifles (1969),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),...,Zeitgeist: Addendum (2008),Zelig (1983),Zero Effect (1998),Zombie Strippers! (2008),Zombieland (2009),Zoolander (2001),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [183]:
ratings_matrix.mean()

title
'Round Midnight (1986)            0.000504
'burbs, The (1989)                0.000560
(500) Days of Summer (2009)       0.000897
1-900 (06) (1994)                 0.001121
10 (1979)                         0.000168
                                    ...   
Zoolander (2001)                  0.003026
eXistenZ (1999)                   0.000953
xXx (2002)                        0.000785
xXx: State of the Union (2005)    0.000280
¡Three Amigos! (1986)             0.002130
Length: 3314, dtype: float64

In [184]:
item_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [185]:
# 영화의 유사도 점수에 해당 영화의 평점을 곱한 값..
# 유사도가 높고, 평점도 높은 영화가 높은 값을 가진다..

item_co_pro = item_sim_df*(ratings_matrix.mean())
item_co_pro.head()

title,'Round Midnight (1986),"'burbs, The (1989)",(500) Days of Summer (2009),1-900 (06) (1994),10 (1979),10 Things I Hate About You (1999),100 Rifles (1969),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),...,Zeitgeist: Addendum (2008),Zelig (1983),Zero Effect (1998),Zombie Strippers! (2008),Zombieland (2009),Zoolander (2001),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,
'Round Midnight (1986),0.000504,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"'burbs, The (1989)",0.000000,0.00056,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(500) Days of Summer (2009),0.000000,0.00000,0.000897,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1-900 (06) (1994),0.000000,0.00000,0.000000,0.001121,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 (1979),0.000000,0.00000,0.000000,0.000000,0.000168,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [186]:
item_co_pro['Godfather, The (1972)'].sort_values(ascending=False)[:10]

title
Godfather, The (1972)                                  0.011769
Hook (1991)                                            0.001068
Mary Poppins (1964)                                    0.000909
'Round Midnight (1986)                                 0.000000
P.S. I Love You (2007)                                 0.000000
Outbreak (1995)                                        0.000000
Outfoxed: Rupert Murdoch's War on Journalism (2004)    0.000000
Outland (1981)                                         0.000000
Outlaw Josey Wales, The (1976)                         0.000000
Outlaw, The (1943)                                     0.000000
Name: Godfather, The (1972), dtype: float64

In [187]:
item_co_pro["Schindler's List (1993)"].sort_values(ascending=False)[1:10]

title
Tomorrow Never Dies (1997)                             0.000817
'Round Midnight (1986)                                 0.000000
Over the Hedge (2006)                                  0.000000
Out of Africa (1985)                                   0.000000
Out of Sight (1998)                                    0.000000
Outbreak (1995)                                        0.000000
Outfoxed: Rupert Murdoch's War on Journalism (2004)    0.000000
Outland (1981)                                         0.000000
Outlaw Josey Wales, The (1976)                         0.000000
Name: Schindler's List (1993), dtype: float64

In [188]:
### 개인화된 아이템 기반 협업 필터링..

In [189]:
ratings_matrix.head(2)

title,'Round Midnight (1986),"'burbs, The (1989)",(500) Days of Summer (2009),1-900 (06) (1994),10 (1979),10 Things I Hate About You (1999),100 Rifles (1969),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),...,Zeitgeist: Addendum (2008),Zelig (1983),Zero Effect (1998),Zombie Strippers! (2008),Zombieland (2009),Zoolander (2001),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [190]:
item_sim_df[:2]

title,'Round Midnight (1986),"'burbs, The (1989)",(500) Days of Summer (2009),1-900 (06) (1994),10 (1979),10 Things I Hate About You (1999),100 Rifles (1969),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),...,Zeitgeist: Addendum (2008),Zelig (1983),Zero Effect (1998),Zombie Strippers! (2008),Zombieland (2009),Zoolander (2001),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,
'Round Midnight (1986),1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"'burbs, The (1989)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [192]:
ratings_matrix.dot(item_sim_df)

# user의 평점점수와 영화의 유사도 점수를 곱하고 더해서 나온 값..
# user가 높은 평점을 준 영화에 대해서 특정 영화가 높은 유사도를 가지면 높은 점수가 부여
# 큰 값(유저의 특정 영화에 대한 평점)과 큰 값(해당 영확 높은 평점의 영화와 유사도)이 곱해져서 큰 값이 나온다..

title,'Round Midnight (1986),"'burbs, The (1989)",(500) Days of Summer (2009),1-900 (06) (1994),10 (1979),10 Things I Hate About You (1999),100 Rifles (1969),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),...,Zeitgeist: Addendum (2008),Zelig (1983),Zero Effect (1998),Zombie Strippers! (2008),Zombieland (2009),Zoolander (2001),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8917,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8919,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8920,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [193]:
def user_based_recomm(item_sim, ratings_matrix, user_id, top_n=10):
    seen_movie_list = seen_movies(user_id)
    
    item_sim_df = pd.DataFrame(item_sim, index=ratings_matrix.columns, columns =ratings_matrix.columns )
    rating_pred_df = ratings_matrix.dot(item_sim_df) 
    unseen_movie_list = set(rating_pred_df.columns).difference(set(seen_movie_list))
    
    return rating_pred_df[unseen_movie_list].loc[user_id].sort_values(ascending=False)[:top_n]

In [194]:
user_based_recomm(item_sim, ratings_matrix, 20, top_n=10)

title
Sex and the City (2008)                               0.0
Fly, The (1986)                                       0.0
Caine Mutiny, The (1954)                              0.0
To Kill a Mockingbird (1962)                          0.0
Zero Effect (1998)                                    0.0
Austin Powers: International Man of Mystery (1997)    0.0
Scent of a Woman (1992)                               0.0
Curly Sue (1991)                                      0.0
Hard Target (1993)                                    0.0
Sex, Lies, and Videotape (1989)                       0.0
Name: 20, dtype: float64